### Import  libraries and read data

In [246]:
# pip install pandas-datareader

### Initial Imports

In [247]:
# from dotenv import load_dotenv
# import alpaca_trade_api as tradeapi
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [248]:
# Universal variable

T1=10
T2=5
stock="AMZN"
benchmark="SPY"
yield10y="^TNX"
crypto="BTC-USD"


end = datetime.date.today()
ed=end.strftime('%Y-%m-%d')

data_begin=(end-pd.DateOffset(365*T1))
st=data_begin.strftime('%Y-%m-%d')

visual_begin=end-pd.DateOffset(365*T2)
vst=visual_begin.strftime('%Y-%m-%d')

In [249]:
# Universal Functions

def n_year_ago(n):
    # to get the date of "n years ago from today"
    n_years_ago=(end-pd.DateOffset(365*n)).strftime('%Y-%m-%d')
    return n_years_ago


### Import data from Yahoo Finance

In [250]:
# Import data from 
def import_data(ticker):
    ticker_df = web.DataReader(ticker, 'yahoo', st, ed)
    ticker_close=round(ticker_df[['Close']],2)
    return ticker_close

def import_all_data(ticker):
    ticker_df = web.DataReader(ticker, 'yahoo', st, ed)
    return ticker_df
     

### Metric Calculation
##### Create function to generate metrics

* 1-year/5-year annualized return 
* 1-year/5-year cumulative return 
* annualized rolling volatility 
* annualized rolling sharpe ratio 


In [251]:
def daily_return(ticker):
    # calculate daily return 
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    return df[['daily_return']]

def yr_a_rt(ticker,years):
    # calculate n-year annualized return
    df=import_data(ticker)
    df[f'{years}yr_a_rt']=(1+df['Close'].pct_change(periods=252*years))**(1/years) - 1
    return df[[f'{years}yr_a_rt']]

def yr_c_rt(ticker,years):
    # calculate n-year cum return
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df[f'{years}yr_c_rt'] = (1 + df['daily_return'])[n_year_ago(years):].cumprod() - 1  
    return df[[f'{years}yr_c_rt']]

def annualized_stddv(ticker):
    # calculate annualized rolling standard deviation:
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df['annualizd_volatility']=df['daily_return'].rolling(252).std()*(252**0.5)
    return df[['annualizd_volatility']]

def annualized_sharpe(ticker):
    # calculate annualized rolling sharpe ratio:
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df['annualizd_sharpe']=(252**0.5)*(df['daily_return'].rolling(252).mean()
                                    /df['daily_return'].rolling(252).std())
    return df[['annualizd_sharpe']]
    
def correlation(data):
    correlation = np.log(data/data.shift())
    return correlation.corr()

### Create Function on Plotting:


In [252]:
# format data into proper structure to be used in chart function

def format_data(ticker1,metric1,ticker2,metric2):
    # metric 1 and 2 are two dataframe generated through metric calculation functions
    metric1['Label']=ticker1
    metric2['Label']=ticker2
    c_df=pd.concat([metric1,metric2])
    
    
    c_df.dropna(inplace=True)

    return c_df

In [253]:
# format data into proper structure to be used in chart function
def plot_lines(df,ylable,title,format):
    fig = px.line(df, x=df.index, y=list(df.columns)[0], color=list(df.columns)[1]
                ,labels={
                        list(df.columns)[0]: ylable
                    }
                ,title=title)
    fig.update_layout(yaxis_tickformat = format)
    return fig


In [254]:
def line_plot(data, label, title):
    fig = px.line(data, x=data.index, y=list(data.columns)[0], labels=label, title=title)
    return fig

In [255]:
def moving_average_with_data(data, stock, ma):
    ma_table = format_data(stock, data, f'{stock} MA{ma}', data.rolling(ma).mean())
    return plot_lines(ma_table, 'Price', f'{stock} Moving Average Plot', '$0')

In [256]:
def moving_average(data, stock, ma):
    return line_plot(data.rolling(ma).mean(), stock, f'{stock} Moving Average Plot')

In [257]:
def kde_plot(data, label):
    return data.plot(kind='kde', label=label, figsize=(15,7)), plt.legend()

In [258]:
def histogram(data, label):
    return data.hist(bins=100, label=label, alpha=0.5, figsize=(15,7)), plt.legend()

### Part 1 - Trading Volume and Closing Price (with MA)

For user to have a overview of the historical price movement and volumn 

Trading Volume

In [259]:
# format data for trading volume
data1= import_all_data(stock)[['Volume']][vst:]/1000000
data1=data1.reset_index()
data1['Date']=data1['Date'].dt.date

data2= import_all_data(benchmark)[['Volume']][vst:]/1000000
data2=data2.reset_index()
data2['Date']=data2['Date'].dt.date



In [260]:

plot2 = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
plot2.add_trace(
    go.Scatter( x=data1['Date'], y=data1["Volume"], name=stock),
    secondary_y=False,
)

plot2.add_trace(
    go.Scatter( x=data2['Date'], y=data2["Volume"], name=benchmark),
    secondary_y=True,
)

# Add figure title
plot2.update_layout(
    title_text=f"Trading Volume: {stock} vs. {benchmark}"
)

# Set x-axis title
plot2.update_xaxes(title_text="Date")

# Set y-axes titles
plot2.update_yaxes(title_text=f"{stock} Trading Volume (MM)", secondary_y=False)
plot2.update_yaxes(title_text=f"{benchmark} Trading Volume (MM)", secondary_y=True)

plot2.show()


In [261]:
# volume1 = import_all_data(stock)[['Volume']][vst:]/1000000

# plot1 = px.line(volume1, x=volume1.index, y=list(volume1.columns)[0],title=f'{stock} Volume (MM) Traded')
# plot1.update_layout(yaxis_tickformat = '0.')
# plot1

In [262]:
# line_plot(import_all_data(stock)[['Volume']][vst:], stock, f'{stock} Volume Traded')

In [263]:
# volume2 = format_data(stock, import_all_data(stock)[['Volume']][vst:]/1000000, benchmark, import_all_data(benchmark)[['Volume']][vst:]/1000000)
# plot2=plot_lines(volume2, 'Volume (MM)', f'{stock} and {benchmark} Volume (MM) Traded', '0.')
# plot2

Closing Price and Moving Average

In [264]:
plot3=moving_average_with_data(import_data(stock)[vst:], stock, 50) 
plot3

In [265]:
plot4=moving_average_with_data(import_data(benchmark)[vst:], benchmark, 50) 
plot4

### Part 2 - Risk and return comparison between selected stock {AMZN} and benchmark {SPY}
direct comparison on the following metrics
* cumulative returns (from 1 year ago, from 5 years ago)
* annualized returns (1-year average, 5-year average) 
* risk (annualized daily return standard deviation)   
* risk-adjusted returns (sharp ratio)

#### Annualized Returns

In [266]:
df2_1=format_data(stock,yr_a_rt(stock,1)[vst:],benchmark,yr_a_rt(benchmark,1)[vst:])
plot5=plot_lines(df2_1,'Annualized Return',f'Rolling 1-year Annualized Return: {stock} vs. {benchmark}','0.2%')
plot5

In [267]:
df2_2=format_data(stock,yr_a_rt(stock,5)[vst:],benchmark,yr_a_rt(benchmark,5)[vst:])
plot6=plot_lines(df2_2,'Annualized Return',f'Rolling 5-year Annualized Return: {stock} vs. {benchmark}','0.2%')
plot6

#### Cumulative Returns (rolling)

In [268]:
df=format_data(stock,yr_c_rt(stock,1)[vst:],benchmark,yr_c_rt(benchmark,1)[vst:])
plot7=plot_lines(df,'Cumulative Return',f'1-year Cumulative Return: {stock} vs. {benchmark}','0.2%')
plot7

In [269]:
df=format_data(stock,yr_c_rt(stock,5)[vst:],benchmark,yr_c_rt(benchmark,5)[vst:])
plot8=plot_lines(df,'Cumulative Return',f'5-year Cumulative Return: {stock} vs. {benchmark}','0.2%')
plot8

#### Risk (std.dev)

In [281]:
df=format_data(stock,annualized_stddv(stock)[vst:],benchmark,annualized_stddv(benchmark)[vst:])
plot9=plot_lines(df,'Volatility',f'Rolling Annualized Volatility: {stock} vs. {benchmark}','0.2%')
plot9

#### Risk-adjusted Return (sharp ratio)
*assuming risk-free rate is 0

In [271]:
df=format_data(stock,annualized_sharpe(stock)[vst:],benchmark,annualized_sharpe(benchmark)[vst:])
plot10=plot_lines(df,'Sharpe Ratio',f'Rolling Annualized Sharpe Ratio: {stock} vs. {benchmark}','0.2')
plot10

### Part 3 - Correlation and Diversification 

These charts should give user/investor an idea of how the stock (in this case "AMZN") is correlated with the broad market (SPY), with interest rate (10-year Bond Yield), and with crypto price (Bitcoin), thus helping them to make decisions from a portfolio diversification point of view

In [272]:
def data_corr(ticker1,ticker2,years):
    # metric 1 and 2 are two dataframe generated through metric calculation functions

    c_df=pd.concat([daily_return(ticker1)[n_year_ago(years):],daily_return(ticker2)[n_year_ago(years):]],axis=1)
    c_df.dropna(inplace=True)

    return c_df.corr().iloc[0,1]

In [273]:
print(data_corr(stock,crypto,1))
print(data_corr(stock,crypto,5))

0.26712221337503933
0.15850746936353102


In [274]:
def correlation_gauge(title,years):
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"{benchmark}"},
        value = data_corr(stock,benchmark,years),
        # delta = {'reference': data_corr(stock,benchmark)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 0}))
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"10yr Treasury Yield"},
        value = data_corr(stock,yield10y,years),
        # delta = {'reference': data_corr(stock,yield10y)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 1}))
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"{crypto}"},
        value = data_corr(stock,crypto,years),
        # delta = {'reference': data_corr(stock,crypto)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 2}))
    
    fig.update_annotations(font=dict(family="Helvetica", size=8))
    
    fig.update_layout(title=title,grid = {'rows': 1, 'columns': 3, 'pattern': "independent"})
    
    return fig

#### (short-term) Correlation during the past 1 year

In [275]:
plot11=correlation_gauge(f"1-year Correlation of {stock} vs.", 1)
plot11


##### (long-term) Correlation based on 5 years of historical price

In [276]:
plot12=correlation_gauge(f"5-year Correlation of {stock} vs.", 5)
plot12

### Dashboard

In [277]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [282]:
#welcome page

welcomepage=pn.Column("## Welcome to Investor Toolkit", f"### Ticker: {stock}")
#Price and Trading Volumn

price_volume_1=pn.Row(plot2)
price_volume_2=pn.Row(plot3,plot4)

#price_volume= pn.Row(pn.Column(plot1,plot3),pn.Column(plot2,plot4))
#Rolling Annualized Return
ar=pn.Row(plot5,plot6)
#Cummulative Return
cr=pn.Row(plot7,plot8)
#Volatility and Sharpe Ratio
vol_sharpe=pn.Row(plot9,plot10)
#Correlations with other sectors
Correlations=pn.Column(plot11,plot12)

In [283]:
tabs = pn.Tabs(
    ("Welcome", welcomepage),
    ("Trading Volume", price_volume_1),
    ("Historical Price", price_volume_2),
    
    
    ("Rolling Annualized Return", ar),
    ("Cummulative Return", cr),
    ("Volatility and Sharpe Ratio", vol_sharpe),
    ("Correlations with other sectors", Correlations),
     
    )

tabs.show()

Launching server at http://localhost:55259
